# Credentials

Load our credentials from Google Drive.

In [ ]:
import json
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
with open('/content/drive/My Drive/Colab/credentials.json') as f:
  credentials = json.loads(f.read())

D4S_USERNAME = credentials.get('d4s').get('username')
D4S_PASSWORD = credentials.get('d4s').get('password')
D4S_USERNAME

# Configuration

In [ ]:
keyword = "avocado recipes" # @param {type:"string"}
location = "United States" # @param {type:"string"}
language = "en" # @param {type:"string"}

# Code

## Boilerplate code for D4S

In [ ]:
from http.client import HTTPSConnection
from base64 import b64encode
from json import loads
from json import dumps

class RestClient:
    domain = "api.dataforseo.com"

    def __init__(self, username, password):
        self.username = username
        self.password = password

    def request(self, path, method, data=None):
        connection = HTTPSConnection(self.domain)
        try:
            base64_bytes = b64encode(
                ("%s:%s" % (self.username, self.password)).encode("ascii")
                ).decode("ascii")
            headers = {'Authorization' : 'Basic %s' %  base64_bytes, 'Content-Encoding' : 'gzip'}
            connection.request(method, path, headers=headers, body=data)
            response = connection.getresponse()
            return loads(response.read().decode())
        finally:
            connection.close()

    def get(self, path):
        return self.request(path, 'GET')

    def post(self, path, data):
        if isinstance(data, str):
            data_str = data
        else:
            data_str = dumps(data)
        return self.request(path, 'POST', data_str)

In [ ]:
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')
today

In [ ]:
client = RestClient(D4S_USERNAME, D4S_PASSWORD)
post_data = dict()
items = []

post_data[len(post_data)] = dict(
    language_code=language,
    location_name=location,
    keyword=keyword
)

response = client.post("/v3/serp/google/organic/live/regular", post_data)

if response["status_code"] == 20000:
  items = response['tasks'][0]["result"][0]["items"]
  for item in items:
    item['keyword'] = keyword
    item['date'] = today
  print(response.get('status_message', 'Could not get status, probably an error'))      
else:
  print("error. Code: %d Message: %s" % (response["status_code"], response["status_message"]))

## Extract results from the D4S API response

In [ ]:
import pandas as pd

df = None
if response.get('status_code', 0) == 20000:
  df = pd.DataFrame.from_dict(items)

df

## Download as an Excel file

In [ ]:
from google.colab import files

filename = f'{keyword}.xlsx'
df.to_excel(filename, index=False)
files.download(filename)